In [ ]:
#from langchain.llms import GooglePalm
import os
from dotenv import load_dotenv
load_dotenv() 
from langchain.utilities import SQLDatabase
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.agents import create_sql_agent


In [54]:
api_key = os.environ["gemini_key"]
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp",google_api_key=api_key, temperature=0)
#llm = ChatOpenAI(model="gpt-4o-mini")
llm

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-exp', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x11ff40c50>, default_metadata=())

In [55]:
llm.invoke("Write me a ballad about LangChain")

AIMessage(content="Okay, here's a ballad about LangChain, with a touch of the dramatic and a sprinkle of tech-speak:\n\n**(The Ballad of the Chain of Language)**\n\nThe world was vast, a sea of text,\nA swirling chaos, complex and vexed.\nFrom ancient lore to modern tweet,\nInformation flowed, both bitter and sweet.\nBut how to grasp, how to discern,\nThe hidden patterns, the lessons to learn?\n\nThen came a whisper, a digital hum,\nOf a framework born, for all to become\nMasters of language, with tools so keen,\nA chain of logic, a powerful machine.\nThey called it LangChain, a name to recall,\nA bridge to knowledge, standing up tall.\n\nIts core was the model, a mind of its own,\nTrained on the data, seeds widely sown.\nBut raw potential, like a diamond rough,\nNeeded a structure, a guiding enough.\nSo LangChain arose, with its modules bright,\nTo shape the responses, and bring forth the light.\n\nFirst came the prompts, the questions we pose,\nCrafted with care, as the story unfolds

In [46]:
conn_string = "sqlite:///sales.db"
db = SQLDatabase.from_uri(conn_string)
db

In [47]:
db.run("SELECT * FROM discounts LIMIT 10;")

'[(0, 21, 1, 10.0), (1, 22, 2, 15.0), (2, 23, 3, 20.0), (3, 24, 4, 5.0), (4, 25, 5, 25.0), (5, 26, 6, 10.0), (6, 27, 7, 30.0), (7, 28, 8, 35.0), (8, 29, 9, 40.0), (9, 30, 10, 45.0)]'

In [48]:
from langchain.chains import create_sql_query_chain
q = "How many t-shirts do we have left for nike in extra small size and white color?"

In [49]:
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many t-shirts do we have left for nike in extra small size and white color?"})
response

'SQLQuery: SELECT "stock_quantity" FROM "t_shirts" WHERE "brand" = \'Nike\' AND "size" = \'XS\' AND "color" = \'White\' LIMIT 5;'

In [50]:
response.split(':')[1]

' SELECT "stock_quantity" FROM "t_shirts" WHERE "brand" = \'Nike\' AND "size" = \'XS\' AND "color" = \'White\' LIMIT 5;'

In [51]:
db.run(response.split(':')[1])

'[(25,)]'

In [52]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [56]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x13c879390>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x13c879390>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x13c879390>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)
response = agent.invoke(q)["output"]
response



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:discounts, t_shirtsI should query the schema of the t_shirts table to see the columns available.
Action: sql_db_schema
Action Input: t_shirts
CREATE TABLE t_shirts (
	"index" BIGINT, 
	t_shirt_id BIGINT, 
	brand TEXT, 
	color TEXT, 
	size TEXT, 
	price BIGINT, 
	stock_quantity BIGINT
)

/*
3 rows from t_shirts table:
index	t_shirt_id	brand	color	size	price	stock_quantity
0	1	Adidas	Black	M	44	57
1	2	Van Huesen	White	S	48	65
2	3	Levi	Black	XS	39	33
*/I should query the t_shirts table to find the number of t-shirts that match the criteria.
Action: sql_db_query_checker
Action Input: SELECT stock_quantity FROM t_shirts WHERE brand = "Nike" AND color = "White" AND size = "XS"```sql
SELECT stock_quantity FROM t_shirts WHERE brand = "Nike" AND color = "White" AND size = "XS"
```I should execute the query to find the number of t-shirts that match the criteria.
Action: sql_db_query
Action Input: SELECT stock_qu

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


I should execute the query to find the number of t-shirts that match the criteria.
Action: sql_db_query
Action Input: SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'[(25,)]I now know the final answer
Final Answer: 25

> Finished chain.


'25'

In [66]:
q2 = "How much is the price of the inventory for all small size t-shirts?"
agent.invoke(q2)["output"]



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:discounts, t_shirtsThe tables are discounts and t_shirts. I should query the schema of the t_shirts table.
Action: sql_db_schema
Action Input: t_shirts
CREATE TABLE t_shirts (
	"index" BIGINT, 
	t_shirt_id BIGINT, 
	brand TEXT, 
	color TEXT, 
	size TEXT, 
	price BIGINT, 
	stock_quantity BIGINT
)

/*
3 rows from t_shirts table:
index	t_shirt_id	brand	color	size	price	stock_quantity
0	1	Adidas	Black	M	44	57
1	2	Van Huesen	White	S	48	65
2	3	Levi	Black	XS	39	33
*/I should query the t_shirts table to find the price and stock quantity of all small size t-shirts. Then I should multiply the price and stock quantity to find the inventory value. Finally, I should sum the inventory value for all small size t-shirts.
Action: sql_db_query
Action Input: SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'[(25416,)]I now know the final answer
Final Answer: The total price of the inventory for all small s

'The total price of the inventory for all small size t-shirts is 25416.'

In [67]:
q3 = "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"
agent.invoke(q3)["output"]



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:discounts, t_shirtsI should query the schema of the discounts and t_shirts tables to see what information they contain.
Action: sql_db_schema
Action Input: discounts, t_shirts
CREATE TABLE discounts (
	"index" BIGINT, 
	discount_id BIGINT, 
	t_shirt_id BIGINT, 
	pct_discount FLOAT
)

/*
3 rows from discounts table:
index	discount_id	t_shirt_id	pct_discount
0	21	1	10.0
1	22	2	15.0
2	23	3	20.0
*/


CREATE TABLE t_shirts (
	"index" BIGINT, 
	t_shirt_id BIGINT, 
	brand TEXT, 
	color TEXT, 
	size TEXT, 
	price BIGINT, 
	stock_quantity BIGINT
)

/*
3 rows from t_shirts table:
index	t_shirt_id	brand	color	size	price	stock_quantity
0	1	Adidas	Black	M	44	57
1	2	Van Huesen	White	S	48	65
2	3	Levi	Black	XS	39	33
*/I need to calculate the revenue generated from selling all Levi's t-shirts after applying discounts. I need to join the `t_shirts` and `discounts` tables on `t_shirt_id`, filter for Levi's t-shirts, calc

'2653.5'

In [69]:
q4 = "How much is the total price of the inventory for all S-size t-shirts?"
agent.invoke(q4)["output"]



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:discounts, t_shirtsThe tables are discounts and t_shirts. I should look at the schema of the t_shirts table to see what columns are available.
Action: sql_db_schema
Action Input: t_shirts
CREATE TABLE t_shirts (
	"index" BIGINT, 
	t_shirt_id BIGINT, 
	brand TEXT, 
	color TEXT, 
	size TEXT, 
	price BIGINT, 
	stock_quantity BIGINT
)

/*
3 rows from t_shirts table:
index	t_shirt_id	brand	color	size	price	stock_quantity
0	1	Adidas	Black	M	44	57
1	2	Van Huesen	White	S	48	65
2	3	Levi	Black	XS	39	33
*/I should query the t_shirts table to find the total price of all S-size t-shirts. I need to multiply the price by the stock quantity for each S-size t-shirt and then sum the results.
Action: sql_db_query_checker
Action Input: SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'Action: sql_db_query
Action Input: SELECT SUM(price * stock

'25416'